In [46]:
import numpy as np
import svox2
import torch

path = "./opt/ckpt/udf_lego_8_tv_sur/ckpt.npz"
path = "./opt/ckpt/udf_lego_256_mp_low_lr/ckpt.npz"
path = "./opt/ckpt/udf_lego_16_mp_ex_lr_var_reg/ckpt.npz"
grid = svox2.SparseGrid.load(path, device='cuda')

surface_data = grid.surface_data
alpha_data = grid.density_data
grid.step_id

5000

In [31]:
ls = np.stack(
    np.meshgrid(np.arange(grid.links.shape[0]-1),np.arange(grid.links.shape[1]-1),np.arange(grid.links.shape[2]-1))
    ).reshape(3, -1).T
# ls = np.stack(
#     np.meshgrid(np.arange(64, 128),np.arange(64, 128),np.arange(64, 128))
#     ).reshape(3, -1).T

render_ls = [ls[i] for i in range(ls.shape[0])]

for l in render_ls:
    lx, ly, lz = l
    links000 = grid.links[lx, ly, lz]
    links001 = grid.links[lx, ly, lz + 1]
    links010 = grid.links[lx, ly + 1, lz]
    links011 = grid.links[lx, ly + 1, lz + 1]
    links100 = grid.links[lx + 1, ly, lz]
    links101 = grid.links[lx + 1, ly, lz + 1]
    links110 = grid.links[lx + 1, ly + 1, lz]
    links111 = grid.links[lx + 1, ly + 1, lz + 1]
    alphas = [
        alpha_data[links000,0], 
        alpha_data[links001,0], 
        alpha_data[links010,0], 
        alpha_data[links011,0], 
        alpha_data[links100,0], 
        alpha_data[links101,0], 
        alpha_data[links110,0], 
        alpha_data[links111,0]
        ]
    alphas = torch.sigmoid(torch.stack(alphas)).cpu().detach().numpy()
    if np.min(alphas) >= 0.01:
        print(l)
        print(alphas)
        break


[6 1 5]
[0.9999989  1.         1.         1.         0.36254048 1.
 1.         1.        ]


In [47]:
# l = [2,4,3]
# l = [ 74, 64, 101]
# l = [ 74, 64, 104]




lx, ly, lz = l
links000 = grid.links[lx, ly, lz]
links001 = grid.links[lx, ly, lz + 1]
links010 = grid.links[lx, ly + 1, lz]
links011 = grid.links[lx, ly + 1, lz + 1]
links100 = grid.links[lx + 1, ly, lz]
links101 = grid.links[lx + 1, ly, lz + 1]
links110 = grid.links[lx + 1, ly + 1, lz]
links111 = grid.links[lx + 1, ly + 1, lz + 1]


surfaces = [surface_data[links000,0], surface_data[links001,0], surface_data[links010,0], surface_data[links011,0], surface_data[links100,0], surface_data[links101,0], surface_data[links110,0], surface_data[links111,0]]
alphas = [alpha_data[links000,0], alpha_data[links001,0], alpha_data[links010,0], alpha_data[links011,0], alpha_data[links100,0], alpha_data[links101,0], alpha_data[links110,0], alpha_data[links111,0]]
print(torch.stack(surfaces))
print(torch.stack(alphas))
print(torch.sigmoid(torch.stack(alphas)))

tensor([-0.2808, -0.1471, -0.5816, -1.2880, -0.1003, -1.2476, -0.0587, -0.5634],
       device='cuda:0', grad_fn=<StackBackward0>)
tensor([-6.7964,  2.7451, 25.3549, 20.4087, 11.5054,  5.5626, 18.6356, 27.7375],
       device='cuda:0', grad_fn=<StackBackward0>)
tensor([0.0011, 0.9396, 1.0000, 1.0000, 1.0000, 0.9962, 1.0000, 1.0000],
       device='cuda:0', grad_fn=<SigmoidBackward0>)


In [17]:
surface_data.max()

tensor(221.7026, device='cuda:0', grad_fn=<MaxBackward1>)

In [24]:
torch.sigmoid(grid.density_data).min()

tensor(0.0010, grad_fn=<MinBackward1>)

In [19]:
grid_new = svox2.SparseGrid(reso=grid.links.shape,
                            center=grid.center,
                            radius=grid.radius,
                            use_sphere_bound=False,
                            basis_dim=9,
                            use_z_order=True,
                            device='cuda',
                            basis_reso=32,
                            basis_type=svox2.__dict__['BASIS_TYPE_' + 'sh'.upper()],
                            surface_type=grid.surface_type,
                            surface_init=None)


In [20]:
l = [2,4,3]
l = [2*32,4*32,3*32]
lx, ly, lz = l
links000 = grid.links[lx, ly, lz]
links001 = grid.links[lx, ly, lz + 1]
links010 = grid.links[lx, ly + 1, lz]
links011 = grid.links[lx, ly + 1, lz + 1]
links100 = grid.links[lx + 1, ly, lz]
links101 = grid.links[lx + 1, ly, lz + 1]
links110 = grid.links[lx + 1, ly + 1, lz]
links111 = grid.links[lx + 1, ly + 1, lz + 1]

surface_data = grid_new.surface_data
surfaces = [surface_data[links000,0], surface_data[links001,0], surface_data[links010,0], surface_data[links011,0], surface_data[links100,0], surface_data[links101,0], surface_data[links110,0], surface_data[links111,0]]
surfaces

[tensor(71.5542, device='cuda:0', grad_fn=<SelectBackward0>),
 tensor(71.1126, device='cuda:0', grad_fn=<SelectBackward0>),
 tensor(71.5612, device='cuda:0', grad_fn=<SelectBackward0>),
 tensor(71.1196, device='cuda:0', grad_fn=<SelectBackward0>),
 tensor(70.6612, device='cuda:0', grad_fn=<SelectBackward0>),
 tensor(70.2140, device='cuda:0', grad_fn=<SelectBackward0>),
 tensor(70.6682, device='cuda:0', grad_fn=<SelectBackward0>),
 tensor(70.2211, device='cuda:0', grad_fn=<SelectBackward0>)]

In [33]:
torch.sigmoid(grid_new.density_data)

tensor([[0.0010],
        [0.0010],
        [0.0010],
        ...,
        [0.0010],
        [0.0010],
        [0.0010]], device='cuda:0', grad_fn=<SigmoidBackward0>)

In [26]:
grid_new.density_data

Parameter containing:
tensor([[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]], device='cuda:0', requires_grad=True)

In [31]:
torch.logit(torch.tensor(0.001))

tensor(-6.9068)